# Import Libraries

In [12]:
from google.cloud import bigquery
from google.oauth2 import service_account
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Obtain Data

Create BigQuery client

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= '../Other/Keys/olympics-330304-2824eb690cdc.json'
client = bigquery.Client()

Get world bank wdi dataset

In [5]:
wb_dataset_id = client.dataset("world_bank_wdi", project="bigquery-public-data")
wb_dataset = client.get_dataset(wb_dataset_id)

View tables in world bank dataset

In [6]:
wb_tables = list(client.list_tables(wb_dataset))
for table in wb_tables:
    print(table.table_id)

country_series_definitions
country_summary
footnotes
indicators_data
series_summary
series_time


View world bank indicators_data table

In [7]:
indicator_table_id = wb_dataset_id.table("indicators_data")
indicator_table = client.get_table(indicator_table_id)
[print(item) for item in indicator_table.schema]
client.list_rows(indicator_table, max_results=5).to_dataframe()

SchemaField('country_name', 'STRING', 'NULLABLE', '', (), None)
SchemaField('country_code', 'STRING', 'NULLABLE', '', (), None)
SchemaField('indicator_name', 'STRING', 'NULLABLE', '', (), None)
SchemaField('indicator_code', 'STRING', 'NULLABLE', '', (), None)
SchemaField('value', 'FLOAT', 'NULLABLE', '', (), None)
SchemaField('year', 'INTEGER', 'NULLABLE', '', (), None)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/google/cloud/bigquery/table.py:1860: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  if not self._validate_bqstorage(bqstorage_client, create_bqstorage_client):


,country_name,country_code,indicator_name,indicator_code,value,year
0,Syrian Arab Republic,SYR,Merchandise exports to low- and middle-income ...,TX.VAL.MRCH.R4.ZS,8.595786,1987
1,Syrian Arab Republic,SYR,Merchandise exports to low- and middle-income ...,TX.VAL.MRCH.R4.ZS,19.071772,2004
2,Syrian Arab Republic,SYR,Merchandise exports to low- and middle-income ...,TX.VAL.MRCH.R5.ZS,0.178349,2007
3,Syrian Arab Republic,SYR,Merchandise exports to low- and middle-income ...,TX.VAL.MRCH.R6.ZS,0.286274,1986
4,Syrian Arab Republic,SYR,Merchandise exports to low- and middle-income ...,TX.VAL.MRCH.WR.ZS,39.390389,2008


Query country indicators from indicators_data table

In [11]:
indicators_query = """
        SELECT *
        FROM `bigquery-public-data.world_bank_wdi.indicators_data`
        WHERE indicator_name IN ('Population, total',
                                 'GDP per capita, PPP (constant 2017 international $)',
                                 'Urban population (% of total population)',
                                 'Gini index (World Bank estimate)',
                                 'Current health expenditure (% of GDP)',
                                 'PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)',
                                 'International migrant stock (% of population)',
                                 'Government expenditure on education, total (% of GDP)',
                                 'Population ages 20-24, female (% of female population)',
                                 'Population ages 20-24, male (% of male population)',
                                 'Population ages 25-29, female (% of female population)',
                                 'Population ages 25-29, male (% of male population)',
                                 'Population ages 30-34, female (% of female population)',
                                 'Population ages 30-34, male (% of male population)')
        AND YEAR >= 2010"""
indicators_query_job = client.query(indicators_query)
indicators_df = indicators_query_job.to_dataframe()
indicators_df.to_csv('Data/indicators.csv', index=False)

Scrape 2020 olympics national medal counts and save to csv

In [13]:
response_headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
url  = 'https://www.cbssports.com/olympics/news/tokyo-olympics-final-medal-table-usa-leads-in-in-gold-silver-bronze-and-overall-with-more-than-100-medals/'
response = requests.get(url, headers = response_headers, timeout=5)